In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import ast
import itertools
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/kaggle/input/hackathon/final_results.csv')

In [3]:
df.head()

,Unnamed: 0,name,term,facts,decision_type,first_party,second_party,first_party_winner,issue_area,facts_len,...,llama_prediction,llama_reasoning,llama_consistency_predictions,llama_consistency_reasonings,llama_consistency_scores,deepseek_prediction,deepseek_reasoning,deepseek_consistency_predictions,deepseek_consistency_reasonings,deepseek_consistency_scores
0,464,School Committee of the Town of Burlington v. ...,1984,<p>Under the provisions of the Education of th...,majority opinion,"School Committee of Burlington, Massachusetts,...",Department of Education of the Commonwealth of...,False,Civil Rights,3039,...,1,The Education of the Handicapped Act requires...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","["" The Education of the Handicapped Act requir...",100.0,0,The reasoning for this decision is based on t...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","["" The reasoning for this decision is based on...",100.0
1,607,Graham v. Connor,1988,"<p>On November 12, 1984, Dethorne Graham, a di...",majority opinion,Dethorne Graham,M.S. Connor,True,Civil Rights,2643,...,1,I rule in favor of Dethorne Graham because th...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[' I rule in favor of Dethorne Graham because ...,100.0,1,The facts of the case demonstrate that Dethor...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[' The facts of the case demonstrate that Deth...,100.0
2,1593,Sosa v. Alvarez-Machain,2003,<p>A U.S. Drug Enforcement Agency (DEA) specia...,majority opinion,Jose Francisco Sosa,"Humberto Alvarez-Machain, et al.",True,Economic Activity,2549,...,0,The case presented involves Jose Francisco So...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","["" The case centers around Humberto Alvarez-Ma...",100.0,0,The Alien Tort Statute (ATS) provides federal...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","["" The Alien Tort Statute (ATS) provides feder...",100.0
3,1810,Jones v. Bock,2006,<p>Congress passed the Prisoner Litigation Ref...,majority opinion,Lorenzo L. Jones,"Barbara Bock, Warden, et al.",True,Criminal Procedure,2758,...,0,The Prisoner Litigation Reform Act (PLRA) req...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","["" The Prisoner Litigation Reform Act (PLRA) r...",100.0,0,The Prisoner Litigation Reform Act (PLRA) req...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","["" The Prisoner Litigation Reform Act (PLRA) r...",100.0
4,1876,Boumediene v. Bush,2007,<p>In 2002 Lakhdar Boumediene and five other A...,majority opinion,Lakhdar Boumediene et al.,"George W. Bush, President of the United States...",True,Criminal Procedure,2608,...,1,The Suspension Clause of the Constitution sta...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","["" The Suspension Clause of the Constitution s...",100.0,1,The Supreme Court ruled in favor of Lakhdar B...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[' The Supreme Court ruled in favor of Lakhdar...,100.0


In [4]:
list(df)

['Unnamed: 0',
 'name',
 'term',
 'facts',
 'decision_type',
 'first_party',
 'second_party',
 'first_party_winner',
 'issue_area',
 'facts_len',
 'facts_cleaned',
 'first_party_winner_binary',
 'llama_prediction',
 'llama_reasoning',
 'llama_consistency_predictions',
 'llama_consistency_reasonings',
 'llama_consistency_scores',
 'deepseek_prediction',
 'deepseek_reasoning',
 'deepseek_consistency_predictions',
 'deepseek_consistency_reasonings',
 'deepseek_consistency_scores']

In [5]:
# Load the model for generating embeddings
model = SentenceTransformer('all-mpnet-base-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
def parse_reasoning_lists(df):
    """
    Parse the reasoning lists from string to actual lists
    """
    for col in ['llama_consistency_reasonings', 'deepseek_consistency_reasonings']:
        df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    
    return df
    
parse_reasoning_lists(df)

,Unnamed: 0,name,term,facts,decision_type,first_party,second_party,first_party_winner,issue_area,facts_len,...,llama_prediction,llama_reasoning,llama_consistency_predictions,llama_consistency_reasonings,llama_consistency_scores,deepseek_prediction,deepseek_reasoning,deepseek_consistency_predictions,deepseek_consistency_reasonings,deepseek_consistency_scores
0,464,School Committee of the Town of Burlington v. ...,1984,<p>Under the provisions of the Education of th...,majority opinion,"School Committee of Burlington, Massachusetts,...",Department of Education of the Commonwealth of...,False,Civil Rights,3039,...,1,The Education of the Handicapped Act requires...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[ The Education of the Handicapped Act require...,100.0,0,The reasoning for this decision is based on t...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[ The reasoning for this decision is based on ...,100.0
1,607,Graham v. Connor,1988,"<p>On November 12, 1984, Dethorne Graham, a di...",majority opinion,Dethorne Graham,M.S. Connor,True,Civil Rights,2643,...,1,I rule in favor of Dethorne Graham because th...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[ I rule in favor of Dethorne Graham because t...,100.0,1,The facts of the case demonstrate that Dethor...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[ The facts of the case demonstrate that Detho...,100.0
2,1593,Sosa v. Alvarez-Machain,2003,<p>A U.S. Drug Enforcement Agency (DEA) specia...,majority opinion,Jose Francisco Sosa,"Humberto Alvarez-Machain, et al.",True,Economic Activity,2549,...,0,The case presented involves Jose Francisco So...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[ The case centers around Humberto Alvarez-Mac...,100.0,0,The Alien Tort Statute (ATS) provides federal...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[ The Alien Tort Statute (ATS) provides federa...,100.0
3,1810,Jones v. Bock,2006,<p>Congress passed the Prisoner Litigation Ref...,majority opinion,Lorenzo L. Jones,"Barbara Bock, Warden, et al.",True,Criminal Procedure,2758,...,0,The Prisoner Litigation Reform Act (PLRA) req...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[ The Prisoner Litigation Reform Act (PLRA) re...,100.0,0,The Prisoner Litigation Reform Act (PLRA) req...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[ The Prisoner Litigation Reform Act (PLRA) re...,100.0
4,1876,Boumediene v. Bush,2007,<p>In 2002 Lakhdar Boumediene and five other A...,majority opinion,Lakhdar Boumediene et al.,"George W. Bush, President of the United States...",True,Criminal Procedure,2608,...,1,The Suspension Clause of the Constitution sta...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[ The Suspension Clause of the Constitution st...,100.0,1,The Supreme Court ruled in favor of Lakhdar B...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[ The Supreme Court ruled in favor of Lakhdar ...,100.0
5,1926,Fitzgerald v. Barnstable School Committee,2008,"<p>In February 2001 Jacqueline Fitzgerald, a k...",majority opinion,"Lisa Fitzgerald, et vir","Barnstable School Committee, et al.",True,Civil Rights,2724,...,0,The school district took prompt and diligent ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[ The school district took prompt and diligent...,100.0,0,The reasoning for this decision is based on t...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[ The reasoning for this decision is based on ...,100.0
6,2174,United States v. Home Concrete & Supply,2011,<p>Plaintiffs Stephen R. Chandler and Robert L...,majority opinion,United States,"Home Concrete & Supply, LLC, et al.",False,Federal Taxation,2982,...,0,The Internal Revenue Code allows the IRS to a...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[ The Internal Revenue Code provides a three-y...,100.0,0,The Court rules against the United States in ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[ The Court rules against the United States in...,100.0
7,2183,Armour 

In [7]:
df['llama_consistency_reasonings']

0     [ The Education of the Handicapped Act require...
1     [ I rule in favor of Dethorne Graham because t...
2     [ The case centers around Humberto Alvarez-Mac...
3     [ The Prisoner Litigation Reform Act (PLRA) re...
4     [ The Suspension Clause of the Constitution st...
5     [ The school district took prompt and diligent...
6     [ The Internal Revenue Code provides a three-y...
7     [ Based on the facts of the case, I would rule...
8     [ Based on the facts of the case, the National...
9     [ The case of Carlos Trevino v. Rick Thaler, D...
10    [ The reasoning behind this decision is that t...
11    [ The decision to rule against Ron Davis, Acti...
12    [ The National Labor Relations Board (NLRB) is...
13    [ The ruling in Griffin v. California, which w...
14    [ I rule in favor of Gregory Welch because the...
15    [ The Fourth Amendment protects against unreas...
16    [ I rule in favor of Terrence Byrd because the...
17    [ The facts of the case suggest that Fane 

In [8]:
df['llama_consistency_reasonings'][0]

[" The Education of the Handicapped Act requires state and local education agencies to provide handicapped children with access to appropriate safeguards, including the right to participate in the development of an individual education program (IEP) and a procedure for review of a proposed IEP. In this case, the Bureau of Special Education Appeals (BSEA) determined that the town's proposed placement was inappropriate and that the Carroll School was best equipped to meet Michael's needs. The BSEA's findings should be given significant weight, and the district court erred in not doing so. Additionally, the Act's purpose is to ensure that handicapped children receive a free and appropriate public education, and requiring the town to pay for Michael's tuition and transportation is consistent with this purpose. The Panicos' decision to place Michael in the Carroll School without the town's consent was reasonable given the town's failure to provide an appropriate IEP, and therefore, the town

In [9]:
def compute_pairwise_similarities(reasoning_list):
    """
    Compute pairwise similarities between all runs
    """
    if not reasoning_list or len(reasoning_list) < 2:
        return [], 0.0
    
    # Generate embeddings for all reasonings
    embeddings = model.encode(reasoning_list, show_progress_bar=False)
    
    # Compute pairwise similarities
    pairs = list(itertools.combinations(range(len(reasoning_list)), 2))
    sim_scores = []
    
    for i, j in pairs:
        sim = cosine_similarity([embeddings[i]], [embeddings[j]])[0][0]
        sim_scores.append(sim)
    
    # Calculate average similarity
    avg_similarity = sum(sim_scores) / len(sim_scores) if sim_scores else 0.0
    
    return sim_scores, avg_similarity

In [10]:
%timeit
def analyze_consistency(df):
    """
    Analyze consistency across runs for each model
    """
    # Create columns to store results
    df['llama_pairwise_sims'] = None
    df['llama_avg_consistency'] = 0.0
    df['deepseek_pairwise_sims'] = None
    df['deepseek_avg_consistency'] = 0.0
    
    # Process each case
    for idx, row in df.iterrows():
        # Process Llama reasonings
        if isinstance(row['llama_consistency_reasonings'], list) and len(row['llama_consistency_reasonings']) > 1:
            sims, avg_sim = compute_pairwise_similarities(row['llama_consistency_reasonings'])
            df.at[idx, 'llama_pairwise_sims'] = sims
            df.at[idx, 'llama_avg_consistency'] = avg_sim
        
        # Process DeepSeek reasonings
        if isinstance(row['deepseek_consistency_reasonings'], list) and len(row['deepseek_consistency_reasonings']) > 1:
            sims, avg_sim = compute_pairwise_similarities(row['deepseek_consistency_reasonings'])
            df.at[idx, 'deepseek_pairwise_sims'] = sims
            df.at[idx, 'deepseek_avg_consistency'] = avg_sim
    
    return df

df = analyze_consistency(df)

In [11]:
df.head()

,Unnamed: 0,name,term,facts,decision_type,first_party,second_party,first_party_winner,issue_area,facts_len,...,llama_consistency_scores,deepseek_prediction,deepseek_reasoning,deepseek_consistency_predictions,deepseek_consistency_reasonings,deepseek_consistency_scores,llama_pairwise_sims,llama_avg_consistency,deepseek_pairwise_sims,deepseek_avg_consistency
0,464,School Committee of the Town of Burlington v. ...,1984,<p>Under the provisions of the Education of th...,majority opinion,"School Committee of Burlington, Massachusetts,...",Department of Education of the Commonwealth of...,False,Civil Rights,3039,...,100.0,0,The reasoning for this decision is based on t...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[ The reasoning for this decision is based on ...,100.0,"[0.95367926, 0.95367926, 0.95367926, 0.9536792...",0.983502,"[0.9668904, 0.98852205, 0.9183328, 0.94312245,...",0.936431
1,607,Graham v. Connor,1988,"<p>On November 12, 1984, Dethorne Graham, a di...",majority opinion,Dethorne Graham,M.S. Connor,True,Civil Rights,2643,...,100.0,1,The facts of the case demonstrate that Dethor...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[ The facts of the case demonstrate that Detho...,100.0,"[1.0, 0.9898444, 0.939271, 0.8671661, 0.939271...",0.934155,"[0.9626409, 0.9387245, 0.8028759, 0.9354675, 0...",0.925849
2,1593,Sosa v. Alvarez-Machain,2003,<p>A U.S. Drug Enforcement Agency (DEA) specia...,majority opinion,Jose Francisco Sosa,"Humberto Alvarez-Machain, et al.",True,Economic Activity,2549,...,100.0,0,The Alien Tort Statute (ATS) provides federal...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[ The Alien Tort Statute (ATS) provides federa...,100.0,"[1.0000001, 1.0000001, 0.9341763, 1.0000001, 1...",0.966119,"[0.93738645, 0.75960124, 0.76960766, 0.7681567...",0.905918
3,1810,Jones v. Bock,2006,<p>Congress passed the Prisoner Litigation Ref...,majority opinion,Lorenzo L. Jones,"Barbara Bock, Warden, et al.",True,Criminal Procedure,2758,...,100.0,0,The Prisoner Litigation Reform Act (PLRA) req...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[ The Prisoner Litigation Reform Act (PLRA) re...,100.0,"[1.0, 0.888085, 0.9337025, 1.0, 1.0, 1.0, 1.0,...",0.951282,"[0.89294267, 0.85992765, 0.9377768, 0.91031945...",0.910570
4,1876,Boumediene v. Bush,2007,<p>In 2002 Lakhdar Boumediene and five other A...,majority opinion,Lakhdar Boumediene et al.,"George W. Bush, President of the United States...",True,Criminal Procedure,2608,...,100.0,1,The Supreme Court ruled in favor of Lakhdar B...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[ The Supreme Court ruled in favor of Lakhdar ...,100.0,"[1.0000002, 1.0000002, 1.0000002, 1.0000002, 1...",0.983872,"[0.9620358, 0.9651787, 0.9648342, 0.97656316, ...",0.954120


In [12]:
'''
The formula for calculating combinations is:
nCr = n! / (r! * (n-r)!)
where n is the total number of items and r is the number of items to choose in each combination.
In your case, n = 20 (the number of items in the column), and r = 2 (because you are taking combinations of 2 items).
So, the calculation is:
20C2 = 20! / (2! * 18!) = (20 * 19) / (2 * 1) = 190
'''

len(df['deepseek_pairwise_sims'][1])

190

In [13]:
def visualize_consistency_distribution(df):
    """
    Create a histogram comparing consistency distributions between models
    """
    # Prepare data for plotting
    llama_avg = df['llama_avg_consistency'].dropna()
    deepseek_avg = df['deepseek_avg_consistency'].dropna()
    
    # Create figure
    fig = go.Figure()
    
    # Add Llama histogram
    fig.add_trace(go.Histogram(
        x=llama_avg,
        name='Llama',
        opacity=0.7,
        marker_color='#9467bd',
        xbins=dict(start=0, end=1, size=0.05),
        histnorm='probability density'
    ))
    
    # Add DeepSeek histogram
    fig.add_trace(go.Histogram(
        x=deepseek_avg,
        name='DeepSeek',
        opacity=0.7,
        marker_color='#1f77b4',
        xbins=dict(start=0, end=1, size=0.05),
        histnorm='probability density'
    ))
    
    # Update layout
    fig.update_layout(
        title='Distribution of Consistency Scores Between Models',
        xaxis_title='Average Pairwise Similarity (Consistency)',
        yaxis_title='Probability Density',
        template='plotly_white',
        font=dict(family='Arial', size=14),
        title_font=dict(size=18),
        barmode='overlay',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        margin=dict(l=50, r=50, t=80, b=50),
    )
    
    # Add mean lines
    fig.add_vline(x=llama_avg.mean(), line_dash="dash", line_color="#9467bd", 
                  annotation_text=f"LLama Mean: {llama_avg.mean():.3f}", 
                  annotation_position="top right")
    
    fig.add_vline(x=deepseek_avg.mean(), line_dash="dash", line_color="#1f77b4", 
                  annotation_text=f"DeepSeek Mean: {deepseek_avg.mean():.3f}", 
                  annotation_position="top left")
    
    return fig

visualize_consistency_distribution(df)

In [14]:
df['issue_area'].value_counts()

issue_area
Civil Rights          11
Economic Activity      9
Criminal Procedure     7
Judicial Power         6
Federalism             2
First Amendment        2
Federal Taxation       1
Miscellaneous          1
Unions                 1
Name: count, dtype: int64

In [17]:
def visualize_consistency_comparison(df, column_name='issue_area'):
    """
    Create a bar chart comparing consistency between models for each case
    
    Parameters:
    df (DataFrame): DataFrame containing the data
    column_name (str): Column to use for x-axis categories (default: 'issue_area')
    
    Returns:
    Figure: A Plotly figure object
    """
    # Prepare data for plotting
    categories = []
    llama_scores = []
    deepseek_scores = []
    
    for idx, row in df.iterrows():
        # Use the specified column or fallback to an index-based label
        category_name = row.get(column_name, f'Item {idx}')
        categories.append(category_name)
        llama_scores.append(row['llama_avg_consistency'])
        deepseek_scores.append(row['deepseek_avg_consistency'])
    
    # Sort cases by Llama consistency
    sorted_indices = np.argsort(llama_scores)
    sorted_categories = [categories[i] for i in sorted_indices]
    sorted_llama = [llama_scores[i] for i in sorted_indices]
    sorted_deepseek = [deepseek_scores[i] for i in sorted_indices]
    
    # Create figure
    fig = go.Figure()
    
    # Add Llama bars
    fig.add_trace(go.Bar(
        x=sorted_categories,
        y=sorted_llama,
        name='Llama',
        marker_color='#9467bd',
        opacity=0.7
    ))
    
    # Add DeepSeek bars
    fig.add_trace(go.Bar(
        x=sorted_categories,
        y=sorted_deepseek,
        name='DeepSeek',
        marker_color='#1f77b4',
        opacity=0.7
    ))
    
    # Dynamic title based on column used
    title_text = f'Consistency Comparison by {column_name.replace("_", " ").title()}'
    
    # Update layout
    fig.update_layout(
        title=title_text,
        xaxis_title=column_name.replace("_", " ").title(),
        yaxis_title='Average Pairwise Similarity',
        template='plotly_white',
        font=dict(family='Arial', size=14),
        title_font=dict(size=18),
        barmode='group',
        xaxis=dict(
            tickangle=45,
            tickfont=dict(size=10),
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        margin=dict(l=50, r=50, t=80, b=150),
        height=600
    )
    
    return fig

visualize_consistency_comparison(df, column_name='issue_area')

In [18]:
visualize_consistency_comparison(df, column_name='name')